In [3]:
import json
import requests
from pprint import pprint
import openai
import os
from langchain.llms.openai import AzureOpenAI
import random
USE_AZURE_OPENAI = os.environ.get("USE_AZURE_OPENAI") or True
AZURE_OPENAI_GPT_DEPLOYMENT = os.environ.get(
    "AZURE_OPENAI_GPT_DEPLOYMENT") or "text-davinci-003"
AZURE_OPENAI_CHATGPT_DEPLOYMENT = os.environ.get(
    "AZURE_OPENAI_CHATGPT_DEPLOYMENT") or "gpt-35-turbo"
AZURE_OPENAI_SERVICE_1 = os.environ.get("AZURE_OPENAI_SERVICE_1") or "apac-csu-aoai-eastus-2"
AZURE_OPENAI_SERVICE_1_KEY = os.environ.get("AZURE_OPENAI_SERVICE_1_KEY") or "7cc6a0ee5c43488a8126e9cb3c4ab4ae"
AZURE_OPENAI_SERVICE_2 = os.environ.get("AZURE_OPENAI_SERVICE_2") or "apac-csu-aoai-southcentralus-2"
AZURE_OPENAI_SERVICE_2_KEY = os.environ.get("AZURE_OPENAI_SERVICE_2_KEY") or "340bcce4c1cf4b5a9668e4f2401e200f"
AZURE_OPENAI_SERVICE_3 = os.environ.get("AZURE_OPENAI_SERVICE_3") or "apac-csu-aoai-westeu-2"
AZURE_OPENAI_SERVICE_3_KEY = os.environ.get("AZURE_OPENAI_SERVICE_3_KEY") or "9aa63d7f87f74681969ec774c91e6e03"
AZURE_OPENAI_DEFAULT_TEMP = os.environ.get("AZURE_OPENAI_DEFAULT_TEMP") or 0

In [4]:
api_endpoints = [
    {"base_url": f"https://{AZURE_OPENAI_SERVICE_1}.openai.azure.com/deployments", "key": f"{AZURE_OPENAI_SERVICE_1_KEY}"},
    {"base_url": f"https://{AZURE_OPENAI_SERVICE_2}.openai.azure.com/deployments", "key": f"{AZURE_OPENAI_SERVICE_2_KEY}"},
    {"base_url": f"https://{AZURE_OPENAI_SERVICE_3}.openai.azure.com/deployments", "key": f"{AZURE_OPENAI_SERVICE_3_KEY}"},
]

# Define a function to get a random endpoint from the list
def get_random_endpoint():
    return random.choice(api_endpoints)

In [9]:
get_random_endpoint()

{'base_url': 'https://apac-csu-aoai-southcentralus-2.openai.azure.com/deployments',
 'key': '340bcce4c1cf4b5a9668e4f2401e200f'}

In [11]:
global index
index = 0
if USE_AZURE_OPENAI:
    # Used by the OpenAI SDK to authenticate
    openai.api_type = "azure"
    # openai.api_base = f"https://{AZURE_OPENAI_SERVICE}.openai.azure.com"
    openai.api_version = "2023-03-15-preview"
    llm_gpt_01 = AzureOpenAI(deployment_name=f"https://{AZURE_OPENAI_SERVICE_1}.openai.azure.com/deployments/{AZURE_OPENAI_GPT_DEPLOYMENT}", temperature=f"{AZURE_OPENAI_DEFAULT_TEMP}", openai_api_key=AZURE_OPENAI_SERVICE_1_KEY) # type: ignore
    llm_gpt_02 = AzureOpenAI(deployment_name=f"https://{AZURE_OPENAI_SERVICE_2}.openai.azure.com/deployments/{AZURE_OPENAI_GPT_DEPLOYMENT}", temperature=f"{AZURE_OPENAI_DEFAULT_TEMP}", openai_api_key=AZURE_OPENAI_SERVICE_2_KEY) # type: ignore
    llm_gpt_03 = AzureOpenAI(deployment_name=f"https://{AZURE_OPENAI_SERVICE_3}.openai.azure.com/deployments/{AZURE_OPENAI_GPT_DEPLOYMENT}", temperature=f"{AZURE_OPENAI_DEFAULT_TEMP}", openai_api_key=AZURE_OPENAI_SERVICE_3_KEY) # type: ignore

    llm_chat_gpt_01 = AzureOpenAI(deployment_name=f"https://{AZURE_OPENAI_SERVICE_1}.openai.azure.com/deployments/{AZURE_OPENAI_CHATGPT_DEPLOYMENT}", temperature=f"{AZURE_OPENAI_DEFAULT_TEMP}", openai_api_key=AZURE_OPENAI_SERVICE_1_KEY) # type: ignore
    llm_chat_gpt_02 = AzureOpenAI(deployment_name=f"https://{AZURE_OPENAI_SERVICE_2}.openai.azure.com/deployments/{AZURE_OPENAI_CHATGPT_DEPLOYMENT}", temperature=f"{AZURE_OPENAI_DEFAULT_TEMP}", openai_api_key=AZURE_OPENAI_SERVICE_2_KEY) # type: ignore
    llm_chat_gpt_03 = AzureOpenAI(deployment_name=f"https://{AZURE_OPENAI_SERVICE_3}.openai.azure.com/deployments/{AZURE_OPENAI_CHATGPT_DEPLOYMENT}", temperature=f"{AZURE_OPENAI_DEFAULT_TEMP}", openai_api_key=AZURE_OPENAI_SERVICE_3_KEY) # type: ignore

    llm_gpt = [llm_gpt_01, llm_gpt_02, llm_gpt_03]
    llm_chat_gpt = [llm_chat_gpt_01, llm_chat_gpt_02, llm_chat_gpt_03]

    # round robin across the three services from the list above
    def select_client(clients, index):
        """
        Selects a client from the list of clients based on round-robin scheduling.
        """
        return clients[index % len(clients)]
    
    gpt_client = select_client(llm_gpt, index)
    chat_client = select_client(llm_chat_gpt, index)
    index += 1
    
else:
    pass

In [2]:
# Define the names for the data source, skillset, index and indexer
datasource_name = "cogsrch-py-datasource"
skillset_name = "cogsrch-py-skillset"
index_name = "cogsrch-py-index"
indexer_name = "cogsrch-py-indexer"

In [3]:
# Setup the endpoint
endpoint = 'https://gptkb-y2nmeuebipp4i.search.windows.net/'
headers = {'Content-Type': 'application/json',
           'api-key': 'OlNKmdGhVIHW1obCUGltQ4uGYgwk85AsvJ64FO25sKAzSeCuTbYz'}
params = {
    'api-version': '2020-06-30'
}

In [4]:
# Create a data source
datasourceConnectionString = "DefaultEndpointsProtocol=https;AccountName=demofailover001;AccountKey=pdvCMTqygniVz5f+9AK+/qiDZzXYQ8iq+XRPnYlT84oHz4K6xteyyTKvyu/H7T0Vw+U9lS/ynIlH+ASts9eFXg==;EndpointSuffix=core.windows.net"
datasource_payload = {
    "name": datasource_name,
    "description": "Demo files to demonstrate cognitive search capabilities.",
    "type": "azureblob",
    "credentials": {
        "connectionString": datasourceConnectionString
    },
    "container": {
        "name": "container1"
    }
}
r = requests.put(endpoint + "/datasources/" + datasource_name,
                 data=json.dumps(datasource_payload), headers=headers, params=params)
print(r.status_code)

201


In [5]:
# Create a skillset
skillset_payload = {
    "name": skillset_name,
    "description":
    "Extract entities, detect language and extract key-phrases",
    "skills":
    [
        {
            "@odata.type": "#Microsoft.Skills.Text.V3.EntityRecognitionSkill",
            "categories": ["Organization"],
            "defaultLanguageCode": "en",
            "inputs": [
                {
                    "name": "text", 
                    "source": "/document/content"
                }
            ],
            "outputs": [
                {
                    "name": "organizations", 
                    "targetName": "organizations"
                }
            ]
        },
        {
            "@odata.type": "#Microsoft.Skills.Text.LanguageDetectionSkill",
            "inputs": [
                {
                    "name": "text", 
                    "source": "/document/content"
                }
            ],
            "outputs": [
                {
                    "name": "languageCode",
                    "targetName": "languageCode"
                }
            ]
        },
        {
            "@odata.type": "#Microsoft.Skills.Text.SplitSkill",
            "textSplitMode": "pages",
            "maximumPageLength": 4000,
            "inputs": [
                {
                    "name": "text",
                    "source": "/document/content"
                },
                {
                    "name": "languageCode",
                    "source": "/document/languageCode"
                }
            ],
            "outputs": [
                {
                    "name": "textItems",
                    "targetName": "pages"
                }
            ]
        },
        {
            "@odata.type": "#Microsoft.Skills.Text.KeyPhraseExtractionSkill",
            "context": "/document/pages/*",
            "inputs": [
                {
                    "name": "text", 
                    "source": "/document/pages/*"
                },
                {
                    "name": "languageCode", 
                    "source": "/document/languageCode"
                }
            ],
            "outputs": [
                {
                    "name": "keyPhrases",
                    "targetName": "keyPhrases"
                }
            ]
        }
    ]
}

r = requests.put(endpoint + "/skillsets/" + skillset_name,
                 data=json.dumps(skillset_payload), headers=headers, params=params)
print(r.status_code)

201


In [6]:
# Create an index
index_payload = {
    "name": index_name,
    "fields": [
        {
            "name": "id",
            "type": "Edm.String",
            "key": "true",
            "searchable": "true",
            "filterable": "false",
            "facetable": "false",
            "sortable": "true"
        },
        {
            "name": "content",
            "type": "Edm.String",
            "sortable": "false",
            "searchable": "true",
            "filterable": "false",
            "facetable": "false"
        },
        {
            "name": "languageCode",
            "type": "Edm.String",
            "searchable": "true",
            "filterable": "false",
            "facetable": "false"
        },
        {
            "name": "keyPhrases",
            "type": "Collection(Edm.String)",
            "searchable": "true",
            "filterable": "false",
            "facetable": "false"
        },
        {
            "name": "organizations",
            "type": "Collection(Edm.String)",
            "searchable": "true",
            "sortable": "false",
            "filterable": "false",
            "facetable": "false"
        }
    ]
}

r = requests.put(endpoint + "/indexes/" + index_name,
                 data=json.dumps(index_payload), headers=headers, params=params)
print(r.status_code)

201


In [8]:
# Create an indexer
indexer_payload = {
    "name": indexer_name,
    "dataSourceName": datasource_name,
    "targetIndexName": index_name,
    "skillsetName": skillset_name,
    "fieldMappings": [
        {
            "sourceFieldName": "metadata_storage_path",
            "targetFieldName": "id",
            "mappingFunction":
            {"name": "base64Encode"}
        },
        {
            "sourceFieldName": "content",
            "targetFieldName": "content"
        }
    ],
    "outputFieldMappings":
    [
        {
            "sourceFieldName": "/document/organizations",
            "targetFieldName": "organizations"
        },
        {
            "sourceFieldName": "/document/pages/*/keyPhrases/*",
            "targetFieldName": "keyPhrases"
        },
        {
            "sourceFieldName": "/document/languageCode",
            "targetFieldName": "languageCode"
        }
    ],
    "parameters":
    {
        "maxFailedItems": -1,
        "maxFailedItemsPerBatch": -1,
        "configuration":
        {
            "dataToExtract": "contentAndMetadata",
            "imageAction": "generateNormalizedImages"
        }
    }
}

r = requests.put(endpoint + "/indexers/" + indexer_name,
                 data=json.dumps(indexer_payload), headers=headers, params=params)
print(r.status_code)

201
